In [10]:
import pandas as pd
import numpy as np
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import datetime

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

from statsmodels.formula.api  import ols
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn import metrics


In [8]:

#Given the lack of computing resources, this analysis takes 100,000 articles as an example. 

topicdata = pd.read_csv(r'/Users/cairo/Google Drive/wechat data/TopicOutcomeAll20Topic.csv', nrows = 100000,sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
#topicdata = pd.read_csv(r'C:/Users/Junhao/Google Drive/wechat data/TopicOutcomeAll20Topic.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
    

In [11]:
topicdata.head()

,name,account,author,type,title,summary,likeCount,clicksCount,url,publicTime,...,topic10,topic11,topic12,topic13,topic14,topic15,topic16,topic17,topic18,topic19
0,南京吃喝玩乐,happyinnanjing,南京吃喝玩乐,民生,住宾馆时，怎么知道房间有没有针孔摄像头？,近日，有网友爆料：在入住的宾馆里换拖鞋时，在正对着床的沙发上居然发现了一个针孔摄像头！不仔细...,175,27784,http://mp.weixin.qq.com/s?__biz=MjM5NTkwNTI2MQ...,2015-01-26 12:55:11,...,40.71615,0.05,0.05,0.05,46.502632,0.05,0.05,0.05,0.05,0.05
1,正商阅读,zsyd88,政商阅读,财富,千年至理：爱子七不责,﻿点击上面蓝字“政商阅读”免费关注广受推崇的微信公众号。微信号：zsyd881、对众不责在大...,139,56752,http://mp.weixin.qq.com/s?__biz=MjM5MjEwNzE2NA...,2015-01-24 19:02:07,...,0.05,23.704185,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
2,南京吃喝玩乐,happyinnanjing,南京吃喝玩乐,民生,▶太恶心了，吃火锅前一定要注意，速看！,速看~,113,20763,http://mp.weixin.qq.com/s?__biz=MjM5NTkwNTI2MQ...,2015-01-26 12:55:11,...,0.05,0.05,0.05,0.05,10.045543,0.05,0.05,0.05,0.05,0.05
3,国学精粹与生活艺术,gxjhshys,国学精粹与生活艺术,文化,小时候，你这样二过吗？百分百中枪有木有！,点上方国学精粹与生活艺术免费快速订阅；国学精粹与生活艺术 主编私人微信gxjcyshys这里...,454,87840,http://mp.weixin.qq.com/s?__biz=MzA4OTAzNjMzMQ...,2015-01-18 08:34:10,...,0.05,44.87632,0.05,9.980097,0.05,0.05,20.219387,0.05,0.05,0.05
4,南京吃喝玩乐,happyinnanjing,南京吃喝玩乐,民生,太有用了！看到第二个我就收了!,一. 放个苹果，土豆不发芽二. 用保鲜膜包住根部，可延长香蕉的保存时间三 切水果大全。。。1...,161,17728,http://mp.weixin.qq.com/s?__biz=MjM5NTkwNTI2MQ...,2015-01-27 21:38:15,...,3.7585173,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05


In [78]:
topicdata['name'].nunique() #There are 87 unique accounts in the dataset 

87

In [3]:


###############################################################################################
#prepare data



y = topicdata.likeCount #use likeCount as the dependent variable

#X = topicdata[topicdata.columns[-30:]]
X0 = topicdata.iloc[:,-20:]
X1 = topicdata[["clicksCount", "orderNum", "originalFlag"]]

X = pd.concat([X0, X1], axis=1)
X = sm.add_constant(X) #add a intercept term to the regression
#X.reset_index(drop=True)

#X.index = pd.RangeIndex(len(X.index))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_test = X_test.astype(float)
X_train = X_train.astype(float)
y_train = y_train.astype(float)
y_test = y_test.astype(float)


In [80]:

##################################
#linear regression
##################################


###############################################################################################
#linear regression with all variables
results = sm.OLS(y.astype(float), X.astype(float)).fit()
results.summary().tables[0]


Dep. Variable:,likeCount,R-squared:,0.328
Model:,OLS,Adj. R-squared:,0.328
Method:,Least Squares,F-statistic:,2125.
Date:,"Tue, 28 Sep 2021",Prob (F-statistic):,0.00
Time:,13:31:58,Log-Likelihood:,-8.0027e+05
No. Observations:,100000,AIC:,1.601e+06
Df Residuals:,99976,BIC:,1.601e+06
Df Model:,23,,
Covariance Type:,nonrobust,,


In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

mse_lr=metrics.mean_squared_error(y_test, y_pred)
print("LR MSE: %f" % (mse_lr))

611346.3293772824

In [14]:
#linear regression controlling the fixed effect of influencer accounts 

import statsmodels.formula.api as smf
#df = topicdata[["clicksCount", "orderNum", "originalFlag", "name", "likeCount"]]

df = topicdata[["likeCount","clicksCount", "orderNum", "originalFlag", "name"]]
df.name = pd.Categorical(df.name)
df["name2"] = df.name.cat.codes
df.clicksCount = pd.to_numeric(df.clicksCount)
df.orderNum = pd.Categorical(df.orderNum)
df.originalFlag = pd.Categorical(df.originalFlag)
df.likeCount = pd.to_numeric(df.likeCount)

#df = df.set_index(["name2"], append = True)


FE_ols = smf.ols(formula= "likeCount ~ 1 + orderNum + clicksCount + originalFlag + C(name2)", data = df).fit()
print(FE_ols.summary())



/Users/cairo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-14-db000c77b219>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["name2"] = df.name.cat.codes


                            OLS Regression Results                            
Dep. Variable:              likeCount   R-squared:                       0.472
Model:                            OLS   Adj. R-squared:                  0.471
Method:                 Least Squares   F-statistic:                     939.0
Date:                Tue, 28 Sep 2021   Prob (F-statistic):               0.00
Time:                        15:53:40   Log-Likelihood:            -7.8827e+05
No. Observations:              100000   AIC:                         1.577e+06
Df Residuals:                   99904   BIC:                         1.578e+06
Df Model:                          95                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           -43.4264     11.81

In [17]:
###############################################################################################
#xgboost regression
from xgboost.sklearn import XGBRegressor

xg_reg = XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_train.astype(float),y_train.astype(float))

preds = xg_reg.predict(X_test.astype(float))

xgb_mse = mean_squared_error(y_test.astype(float), preds)
print("Xgboost MSE: %f" % (xgb_mse))




[15:53:53] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
Xgboost MSE: 715101.362978


In [18]:
###########################################################
#regression with decision trees and random forest

from sklearn.tree import DecisionTreeRegressor

dt_regressor = DecisionTreeRegressor(random_state = 0)
dt_regressor.fit(X_train,y_train)
#Predicting using test set 
y_pred = dt_regressor.predict(X_test)

dt_mse=metrics.mean_squared_error(y_test, y_pred)

print("Decision Tree MSE: %f" % (dt_mse))


from sklearn.ensemble import RandomForestRegressor

rf_regressor = RandomForestRegressor(n_estimators = 300 ,  random_state = 0)
rf_regressor.fit(X_train,y_train)
#Predicting the SalePrices using test set 
y_pred = rf_regressor.predict(X_test)

rf_mse=metrics.mean_squared_error(y_test, y_pred)
print("Random Forest MSE: %f" % (rf_mse))


Decision Tree MSE: 686433.536950
Random Forest MSE: 453434.303623


In [12]:
#SVM models
from sklearn.svm import SVR

from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)


regressor= SVR(kernel='linear')
regressor.fit(X_train,y_train)
y_pred_svm=regressor.predict(X_test)
#y_pred_svm = cross_val_predict(regressor, x, y)

svm_mse=metrics.mean_squared_error(y_test, y_pred_svm)
print("SVM MSE: %f" % (svm_mse))


SVM MSE: 777954.776443


In [22]:
###############################################################################################
#######################
#regression in neural networks with keras
######################
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(64, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='relu'))

model.compile(loss='mean_squared_error', 
              optimizer='adam',
              metrics=["mean_squared_error",'mean_absolute_percentage_error']
              
              )
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                250       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                704       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,019
Trainable params: 1,019
Non-trainable params: 0
_________________________________________________________________


In [27]:
history = model.fit(X_train , y_train,
                    epochs=10,
                    verbose=True,
                    #steps_per_epoch=5,            
                    validation_data=(X_test, y_test),
                    batch_size=100
                    )

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

ann_mse = history.history['mean_squared_error'][-1]

Epoch 1/10
800/800 [==============================] - 1s 1ms/step - loss: 524136.7188 - mean_squared_error: 524136.7188 - mean_absolute_percentage_error: 321233856.0000 - val_loss: 621202.5625 - val_mean_squared_error: 621202.5625 - val_mean_absolute_percentage_error: 60218488.0000
Epoch 2/10
800/800 [==============================] - 1s 1ms/step - loss: 502208.5000 - mean_squared_error: 502208.5000 - mean_absolute_percentage_error: 115328560.0000 - val_loss: 601806.5000 - val_mean_squared_error: 601806.5000 - val_mean_absolute_percentage_error: 81940800.0000
Epoch 3/10
800/800 [==============================] - 1s 2ms/step - loss: 492867.2812 - mean_squared_error: 492867.2812 - mean_absolute_percentage_error: 91606432.0000 - val_loss: 595568.8750 - val_mean_squared_error: 595568.8750 - val_mean_absolute_percentage_error: 68541120.0000
Epoch 4/10
800/800 [==============================] - 1s 1ms/step - loss: 487789.7812 - mean_squared_error: 487789.7812 - mean_absolute_percentage_error

In [28]:
#compare all MSE in a table

MSE=[mse_lr,xgb_mse,dt_mse,rf_mse,svm_mse, ann_mse]

col={'Testing MSE':MSE}
models=['Linear Regression','XGBoost','Decision Trees','Random Forest','SVM', "Deep learning neural network"]
df=pd.DataFrame(data=col,index=models)
df

,Testing MSE
Linear Regression,611346.329377
XGBoost,715101.362978
Decision Trees,686433.536950
Random Forest,453434.303623
SVM,777954.776443
Deep learning neural network,476131.218750
